In [6]:
import pandas as pd
import numpy as np
import torch
import dgl

def prepare_graph_data(graphs_file_path, edges_file_path, nodes_file_path, 
                       train_ratio, val_ratio, test_ratio, bidirectional):
    # Reading node and edge data
    node_df = pd.read_csv(nodes_file_path)
    graph_df = pd.read_csv(graphs_file_path)
    edges_df = pd.read_csv(edges_file_path)

    # Initializing feature lists, graph list, and mask lists
    dgl_graphs = []

    # Processing each graph
    for graph_id in graph_df['graph_id']:
        num_nodes = graph_df.loc[graph_df['graph_id'] == graph_id]['num_nodes'].values[0]

        # Initializing node and edge feature arrays and labels
        node_feat = np.zeros((num_nodes, 4))
        edge_feat_proportion = np.zeros((num_nodes, 3), dtype=float)
        room_type_label = np.zeros(num_nodes)

        src_dst_connectivity = edges_df[(edges_df['graph_id'] == graph_id)][['src', 'dst', 'connectivity']]
        connectivity_counts = {0: 0, 1: 0, 2: 0}
        edge_feat = np.zeros((num_nodes, 3))

        # Calculating connectivity counts and edge features
        for _, row in src_dst_connectivity.iterrows():
            src, dst, connectivity = row['src'], row['dst'], row['connectivity']
            edge_feat[src, connectivity] += 1
            connectivity_counts[connectivity] += 1

        # Calculating proportions for edge features
        for src in range(num_nodes):
            for connectivity in range(3):
                total = connectivity_counts[connectivity]
                if total > 0:
                    edge_feat_proportion[src, connectivity] = edge_feat[src, connectivity] / total

        # Assigning node features and labels
        for node_id in range(num_nodes):
            zoning_type = node_df.loc[(node_df['graph_id'] == graph_id) & (node_df['node_id'] == node_id)]['zoning_type'].values[0]
            node_feat[node_id, zoning_type] = 1
            room_type_label[node_id] = node_df.loc[(node_df['graph_id'] == graph_id) & (node_df['node_id'] == node_id)]['room_type'].values[0]

        # Randomly splitting nodes into train, val, and test sets
        num_train = int(train_ratio * num_nodes)
        num_val = int(val_ratio * num_nodes)
        num_test = num_nodes - num_train - num_val

        perm = np.random.permutation(num_nodes)
        train_mask = np.zeros(num_nodes, dtype=bool)
        train_mask[perm[:num_train]] = True
        val_mask = np.zeros(num_nodes, dtype=bool)
        val_mask[perm[num_train:num_train + num_val]] = True
        test_mask = np.zeros(num_nodes, dtype=bool)
        test_mask[perm[num_train + num_val:]] = True

        # Creating the DGL graph
        src_list = src_dst_connectivity["src"].tolist()
        dst_list = src_dst_connectivity["dst"].tolist()
        dgl_graph = dgl.graph((src_list, dst_list), num_nodes=num_nodes)

        feat = torch.cat((torch.tensor(node_feat), torch.tensor(edge_feat_proportion)), dim=1).float()
        label = torch.tensor(room_type_label).to(torch.int64)

        dgl_graph.ndata["feat"] = feat
        dgl_graph.ndata["label"] = label
        dgl_graph.ndata["train_mask"] = torch.tensor(train_mask)
        dgl_graph.ndata["test_mask"] = torch.tensor(test_mask)
        dgl_graph.ndata["val_mask"] = torch.tensor(val_mask)

        if bidirectional:
            dgl_graph = dgl.add_reverse_edges(dgl_graph)        

        dgl_graphs.append(dgl_graph)

    return dgl_graphs



In [7]:
# Usage Example
graphs_file_path = "C:/Users/c22109646/Downloads/graphs_data.csv"
edges_file_path = "C:/Users/c22109646/Downloads/edges_data_1.csv"
nodes_file_path = "C:/Users/c22109646/Downloads/nodes_data_1.csv"


dgl_graphs = prepare_graph_data(graphs_file_path, edges_file_path, nodes_file_path,
                                train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
                                bidirectional=True)

In [10]:
dgl_graphs[:5]

[Graph(num_nodes=18, num_edges=76,
       ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={}),
 Graph(num_nodes=25, num_edges=92,
       ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={}),
 Graph(num_nodes=60, num_edges=236,
       ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={}),
 Graph(num_nodes=19, num

In [28]:
len(dgl_graphs)

4167

In [30]:
dgl_graphs[1].ndata

{'feat': tensor([[0.0000, 1.0000, 0.0000, 0.0000, 0.3333, 0.1471, 0.1667],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0882, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.1667, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.1667, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0588, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0294, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0294, 0.0000],
 

In [15]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

class SAGEModel(nn.Module):
    def __init__(self, in_feats, num_classes, params):
        """
        Initialize the GraphSAGE model.
        
        Parameters:
        in_feats (int): The number of input features.
        num_classes (int): The number of classes for classification.
        params (dict): The hyperparameters for the model, including
                       - 'hidden_units': The number of hidden units.
                       - 'num_hidden_layers': The number of hidden layers.
                       - 'activation': The activation function to be used.
        """
        super(SAGEModel, self).__init__()
        
        self.layers = nn.ModuleList()  # To store the GraphSAGE layers.
        self.batch_norms = nn.ModuleList()  # To store the BatchNorm layers.
        self.activation = params['activation']  # The activation function to be used.
        
        # Input layer: appending the first GraphSAGE layer and its corresponding BatchNorm layer.
        self.layers.append(dglnn.SAGEConv(in_feats, params['hidden_units'], 'pool'))
        self.batch_norms.append(nn.BatchNorm1d(params['hidden_units']))
        
        # Hidden layers: appending the hidden GraphSAGE layers and their corresponding BatchNorm layers.
        for _ in range(params['num_hidden_layers']):
            self.layers.append(dglnn.SAGEConv(params['hidden_units'], params['hidden_units'], 'pool'))
            self.batch_norms.append(nn.BatchNorm1d(params['hidden_units']))
        
        # Output layer: appending the final GraphSAGE layer for classification.
        self.layers.append(dglnn.SAGEConv(params['hidden_units'], num_classes, 'pool'))

    def forward(self, g, in_feat):
        """
        Forward computation on the graph.
        
        Parameters:
        g (DGLGraph): The input DGL graph object.
        in_feat (torch.Tensor): The input feature tensor.
        
        Returns:
        h (torch.Tensor): The output feature tensor.
        """
        h = in_feat  # Initial feature assignment
        # Iterate over all layers except the last one, applying the GraphSAGE layer,
        # followed by batch normalization and activation function.
        for i in range(len(self.layers) - 1):
            h = self.layers[i](g, h)  # GraphSAGE layer
            h = self.batch_norms[i](h)  # Batch normalization
            h = getattr(F, self.activation)(h)  # Activation function
        
        h = self.layers[-1](g, h)  # For the last layer, only the GraphSAGE layer is applied without any activation.
        return h


In [16]:
import torch
import torch.nn.functional as F
import time
from dgl.dataloading import GraphDataLoader

def _TrainClassifier_NC(graphs, model, hparams, batch_size):
    """
    Train a classifier on the provided graph dataset.
    
    Parameters:
    graphs: The input graph dataset.
    model: The model to be trained.
    hparams: Hyperparameters for training.
    batch_size: The batch size for training.
    
    Returns:
    model: The trained model.
    val_acc: The best validation accuracy achieved by the model.
    total_time: The total time spent training the model.
    """
    
    # Creating a DataLoader to batch graphs
    dataloader = GraphDataLoader(graphs, batch_size=batch_size, shuffle=True, drop_last=False)
    
    # Default optimizer
    optimizer = None
    if hparams.optimizer_str.lower() == "adadelta":
        optimizer = torch.optim.Adadelta(model.parameters(), eps=hparams.eps, 
                                            lr=hparams.lr, rho=hparams.rho, weight_decay=hparams.weight_decay)
    elif hparams.optimizer_str.lower() == "adagrad":
        optimizer = torch.optim.Adagrad(model.parameters(), eps=hparams.eps, 
                                            lr=hparams.lr, lr_decay=hparams.lr_decay, weight_decay=hparams.weight_decay)
    elif hparams.optimizer_str.lower() == "adam":
        optimizer = torch.optim.Adam(model.parameters(), amsgrad=hparams.amsgrad, betas=hparams.betas, eps=hparams.eps, 
                                            lr=hparams.lr, maximize=hparams.maximize, weight_decay=hparams.weight_decay)
    
    best_val_acc = 0 
    best_model_state = None  # Store the state_dict of the model with the best validation accuracy
    total_time = 0 
    
    for e in range(hparams.epochs):
        start_time = time.time()  #
        model.train() 
        total_loss = torch.tensor(0.0) 

        for g in dataloader: 
            if not g.ndata: 
                continue
            
            # Extracting features, labels, and train_mask from the current batch
            features, labels, train_mask = g.ndata['feat'], g.ndata['label'], g.ndata['train_mask']
            logits = model(g, features)  # Forward pass
            
            # Compute loss only on the nodes specified by train_mask
            loss = F.cross_entropy(logits[train_mask], labels[train_mask]) if hparams.loss_function.lower() == "cross entropy" else None
            
            # Perform backpropagation and optimizer step
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step()  
            
            total_loss += loss 
        
        # Calculate average loss for the epoch
        average_loss = total_loss / len(dataloader)
        
        
        val_dataloader = GraphDataLoader(graphs, batch_size=32, shuffle=False, drop_last=False)
        model.eval()  
        total_val_acc = 0.0  
        
        # Evaluate model on the validation set
        for val_batched_graph in val_dataloader:
            # Extracting features, labels, and val_mask from the current validation batch
            val_features, val_labels, val_mask = val_batched_graph.ndata['feat'], val_batched_graph.ndata['label'], val_batched_graph.ndata['val_mask']
            val_logits = model(val_batched_graph, val_features) 
            val_pred = val_logits.argmax(1)  
            total_val_acc += (val_pred[val_mask] == val_labels[val_mask]).float().mean().item() 
        
        end_time = time.time() 
        elapsed_time = end_time - start_time 
        val_acc = total_val_acc / len(val_dataloader) 
        
       
        print(f'Epoch {e}, Loss: {average_loss:.4f}, Val Acc: {val_acc:.4f}, Training time: {elapsed_time:.4f} seconds')
        total_time += elapsed_time  # Accumulate the total training time
        
        # If current val_acc is better than the best so far, update best_val_acc and save the model state
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
        
    # Load the best model state
    model.load_state_dict(best_model_state)
    return model, best_val_acc, total_time

In [33]:
import random
from dgl.nn import GINConv, GraphConv, SAGEConv, TAGConv
from topologicpy.DGL import DGL
# Define the Hyper-parameters

optimizer = DGL.Optimizer()
hparams_class = DGL.Hyperparameters(optimizer=optimizer,
                                    model_type="Classifier",
                                    cv_type="Holdout",               
                                    epochs=20,
                                    loss_function="Cross Entropy",
                                    )
    

In [34]:
from sklearn.model_selection import ParameterGrid
from dgl.dataloading import GraphDataLoader


param_grid = {
    'num_hidden_layers': [2,3,4,5],  
    'hidden_units': [16,32,64],  
    'activation': ['relu','tanh'],
    'learning_rate': [0.01, 0.005,0.001],
    'batch_size': [16,32,64]
}

best_acc = 0.0  # Initialize the best accuracy
best_params = None  # Initialize the best parameters

# Iterate over combinations of hyperparameters
for params in ParameterGrid(param_grid):
    print(params)
    optimizer = DGL.Optimizer(lr=params['learning_rate'])
    
    # Create a model instance and set hyperparameters
    model = SAGEModel(in_feats=7, num_classes=9, params=params)  # Ensure that SAGEModel is properly defined and imported
    
    # Train the model
    model, val_acc, total_time = _TrainClassifier_NC(dgl_graphs, 
                                                     model, 
                                                     hparams_class,
                                                     batch_size=params['batch_size'])
    
    # Update the best accuracy and best parameters if the current model performs better on the validation set
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = params
        best_model = model 
    print(f'Total Time: {total_time} seconds')  

# Print the best accuracy and best parameters
print('Best Validation Accuracy: ', best_acc)
print('Best Parameters: ', best_params)

if best_model is not None:
    torch.save(best_model.state_dict(), 'best_model.pth')

{'activation': 'relu', 'batch_size': 16, 'hidden_units': 16, 'learning_rate': 0.01, 'num_hidden_layers': 2}
Epoch 0, Loss: 1.2207, Val Acc: 0.8554, Training time: 3.1040 seconds
Epoch 1, Loss: 0.4232, Val Acc: 0.8870, Training time: 3.1620 seconds
Epoch 2, Loss: 0.3346, Val Acc: 0.8928, Training time: 3.2720 seconds
Epoch 3, Loss: 0.3023, Val Acc: 0.9008, Training time: 3.4030 seconds
Epoch 4, Loss: 0.2767, Val Acc: 0.9066, Training time: 3.4450 seconds
Epoch 5, Loss: 0.2567, Val Acc: 0.9122, Training time: 3.5170 seconds
Epoch 6, Loss: 0.2447, Val Acc: 0.9140, Training time: 3.4700 seconds
Epoch 7, Loss: 0.2361, Val Acc: 0.9150, Training time: 3.4900 seconds
Epoch 8, Loss: 0.2301, Val Acc: 0.9187, Training time: 3.5940 seconds
Epoch 9, Loss: 0.2262, Val Acc: 0.9160, Training time: 3.5320 seconds
Epoch 10, Loss: 0.2226, Val Acc: 0.9226, Training time: 3.5990 seconds
Epoch 11, Loss: 0.2188, Val Acc: 0.9227, Training time: 3.7000 seconds
Epoch 12, Loss: 0.2166, Val Acc: 0.9241, Training 

Epoch 5, Loss: 0.2475, Val Acc: 0.9184, Training time: 5.5850 seconds
Epoch 6, Loss: 0.2406, Val Acc: 0.9177, Training time: 5.2530 seconds
Epoch 7, Loss: 0.2341, Val Acc: 0.9186, Training time: 5.4070 seconds
Epoch 8, Loss: 0.2294, Val Acc: 0.9192, Training time: 5.2670 seconds
Epoch 9, Loss: 0.2267, Val Acc: 0.9210, Training time: 5.3210 seconds
Epoch 10, Loss: 0.2232, Val Acc: 0.9223, Training time: 5.2910 seconds
Epoch 11, Loss: 0.2190, Val Acc: 0.9233, Training time: 5.3470 seconds
Epoch 12, Loss: 0.2188, Val Acc: 0.9226, Training time: 5.2300 seconds
Epoch 13, Loss: 0.2137, Val Acc: 0.9211, Training time: 5.3960 seconds
Epoch 14, Loss: 0.2115, Val Acc: 0.9253, Training time: 5.2960 seconds
Epoch 15, Loss: 0.2100, Val Acc: 0.9257, Training time: 5.5190 seconds
Epoch 16, Loss: 0.2078, Val Acc: 0.9251, Training time: 5.2940 seconds
Epoch 17, Loss: 0.2073, Val Acc: 0.9252, Training time: 5.3960 seconds
Epoch 18, Loss: 0.2060, Val Acc: 0.9251, Training time: 5.3920 seconds
Epoch 19, L

Epoch 11, Loss: 0.2164, Val Acc: 0.9242, Training time: 6.2380 seconds
Epoch 12, Loss: 0.2125, Val Acc: 0.9237, Training time: 6.2810 seconds
Epoch 13, Loss: 0.2106, Val Acc: 0.9250, Training time: 6.4470 seconds
Epoch 14, Loss: 0.2091, Val Acc: 0.9264, Training time: 6.2220 seconds
Epoch 15, Loss: 0.2050, Val Acc: 0.9242, Training time: 6.4320 seconds
Epoch 16, Loss: 0.2053, Val Acc: 0.9274, Training time: 6.2210 seconds
Epoch 17, Loss: 0.2018, Val Acc: 0.9285, Training time: 6.2270 seconds
Epoch 18, Loss: 0.2006, Val Acc: 0.9251, Training time: 6.7390 seconds
Epoch 19, Loss: 0.1997, Val Acc: 0.9271, Training time: 6.6440 seconds
Total Time: 126.46903419494629 seconds
{'activation': 'relu', 'batch_size': 16, 'hidden_units': 16, 'learning_rate': 0.001, 'num_hidden_layers': 5}
Epoch 0, Loss: 1.1563, Val Acc: 0.8208, Training time: 7.5170 seconds
Epoch 1, Loss: 0.4890, Val Acc: 0.8767, Training time: 7.4110 seconds
Epoch 2, Loss: 0.3554, Val Acc: 0.8991, Training time: 7.1980 seconds
Epo

Epoch 17, Loss: 0.1712, Val Acc: 0.9310, Training time: 9.2430 seconds
Epoch 18, Loss: 0.1687, Val Acc: 0.9361, Training time: 8.5650 seconds
Epoch 19, Loss: 0.1665, Val Acc: 0.9352, Training time: 8.5780 seconds
Total Time: 171.99805068969727 seconds
{'activation': 'relu', 'batch_size': 16, 'hidden_units': 32, 'learning_rate': 0.005, 'num_hidden_layers': 2}
Epoch 0, Loss: 0.5900, Val Acc: 0.8953, Training time: 5.0940 seconds
Epoch 1, Loss: 0.2838, Val Acc: 0.9110, Training time: 5.0380 seconds
Epoch 2, Loss: 0.2441, Val Acc: 0.9185, Training time: 5.1840 seconds
Epoch 3, Loss: 0.2283, Val Acc: 0.9208, Training time: 4.9680 seconds
Epoch 4, Loss: 0.2200, Val Acc: 0.9238, Training time: 5.0360 seconds
Epoch 5, Loss: 0.2123, Val Acc: 0.9245, Training time: 5.0110 seconds
Epoch 6, Loss: 0.2087, Val Acc: 0.9271, Training time: 5.0040 seconds
Epoch 7, Loss: 0.2028, Val Acc: 0.9274, Training time: 5.0190 seconds
Epoch 8, Loss: 0.1999, Val Acc: 0.9244, Training time: 5.1040 seconds
Epoch 9, 

Epoch 1, Loss: 0.3273, Val Acc: 0.9075, Training time: 6.1300 seconds
Epoch 2, Loss: 0.2597, Val Acc: 0.9157, Training time: 6.1590 seconds
Epoch 3, Loss: 0.2378, Val Acc: 0.9199, Training time: 6.1020 seconds
Epoch 4, Loss: 0.2237, Val Acc: 0.9223, Training time: 6.0920 seconds
Epoch 5, Loss: 0.2144, Val Acc: 0.9214, Training time: 6.0960 seconds
Epoch 6, Loss: 0.2080, Val Acc: 0.9249, Training time: 6.1490 seconds
Epoch 7, Loss: 0.2024, Val Acc: 0.9266, Training time: 6.1310 seconds
Epoch 8, Loss: 0.1984, Val Acc: 0.9278, Training time: 6.2130 seconds
Epoch 9, Loss: 0.1934, Val Acc: 0.9266, Training time: 6.4380 seconds
Epoch 10, Loss: 0.1895, Val Acc: 0.9278, Training time: 6.1140 seconds
Epoch 11, Loss: 0.1851, Val Acc: 0.9271, Training time: 6.1160 seconds
Epoch 12, Loss: 0.1838, Val Acc: 0.9318, Training time: 6.1900 seconds
Epoch 13, Loss: 0.1809, Val Acc: 0.9309, Training time: 6.1500 seconds
Epoch 14, Loss: 0.1802, Val Acc: 0.9313, Training time: 6.0770 seconds
Epoch 15, Loss:

...

Epoch 2, Loss: 0.5268, Val Acc: 0.8635, Training time: 3.5820 seconds
Epoch 3, Loss: 0.4232, Val Acc: 0.8755, Training time: 3.5540 seconds
Epoch 4, Loss: 0.3714, Val Acc: 0.8891, Training time: 3.5600 seconds
Epoch 5, Loss: 0.3369, Val Acc: 0.8940, Training time: 3.5490 seconds
Epoch 6, Loss: 0.3164, Val Acc: 0.9005, Training time: 3.5480 seconds
Epoch 7, Loss: 0.2957, Val Acc: 0.9047, Training time: 3.5310 seconds
Epoch 8, Loss: 0.2817, Val Acc: 0.9095, Training time: 3.6000 seconds
Epoch 9, Loss: 0.2695, Val Acc: 0.9119, Training time: 3.5790 seconds
Epoch 10, Loss: 0.2609, Val Acc: 0.9128, Training time: 3.5160 seconds
Epoch 11, Loss: 0.2515, Val Acc: 0.9158, Training time: 3.5670 seconds
Epoch 12, Loss: 0.2466, Val Acc: 0.9160, Training time: 3.5610 seconds
Epoch 13, Loss: 0.2433, Val Acc: 0.9193, Training time: 3.5980 seconds
Epoch 14, Loss: 0.2412, Val Acc: 0.9175, Training time: 3.5450 seconds
Epoch 15, Loss: 0.2353, Val Acc: 0.9186, Training time: 3.5560 seconds
Epoch 16, Loss

Epoch 8, Loss: 0.2159, Val Acc: 0.9240, Training time: 4.8530 seconds
Epoch 9, Loss: 0.2097, Val Acc: 0.9256, Training time: 4.8800 seconds
Epoch 10, Loss: 0.2035, Val Acc: 0.9254, Training time: 4.8810 seconds
Epoch 11, Loss: 0.1995, Val Acc: 0.9248, Training time: 4.8930 seconds
Epoch 12, Loss: 0.1963, Val Acc: 0.9255, Training time: 4.8950 seconds
Epoch 13, Loss: 0.1933, Val Acc: 0.9260, Training time: 4.9010 seconds
Epoch 14, Loss: 0.1925, Val Acc: 0.9271, Training time: 4.9970 seconds
Epoch 15, Loss: 0.1912, Val Acc: 0.9280, Training time: 4.9790 seconds
Epoch 16, Loss: 0.1832, Val Acc: 0.9296, Training time: 4.8890 seconds
Epoch 17, Loss: 0.1797, Val Acc: 0.9288, Training time: 4.9030 seconds
Epoch 18, Loss: 0.1782, Val Acc: 0.9315, Training time: 4.8570 seconds
Epoch 19, Loss: 0.1784, Val Acc: 0.9292, Training time: 4.9390 seconds
Total Time: 98.24702024459839 seconds
{'activation': 'tanh', 'batch_size': 64, 'hidden_units': 32, 'learning_rate': 0.005, 'num_hidden_layers': 2}
Epo

Epoch 14, Loss: 0.2077, Val Acc: 0.9263, Training time: 2.9530 seconds
Epoch 15, Loss: 0.2059, Val Acc: 0.9269, Training time: 2.9640 seconds
Epoch 16, Loss: 0.2041, Val Acc: 0.9282, Training time: 3.3670 seconds
Epoch 17, Loss: 0.2017, Val Acc: 0.9272, Training time: 3.7100 seconds
Epoch 18, Loss: 0.1995, Val Acc: 0.9282, Training time: 2.9300 seconds
Epoch 19, Loss: 0.1986, Val Acc: 0.9260, Training time: 2.9350 seconds
Total Time: 60.293020248413086 seconds
{'activation': 'tanh', 'batch_size': 64, 'hidden_units': 32, 'learning_rate': 0.001, 'num_hidden_layers': 3}
Epoch 0, Loss: 1.1205, Val Acc: 0.8612, Training time: 3.6140 seconds
Epoch 1, Loss: 0.3919, Val Acc: 0.8891, Training time: 3.6560 seconds
Epoch 2, Loss: 0.3177, Val Acc: 0.9005, Training time: 3.5990 seconds
Epoch 3, Loss: 0.2846, Val Acc: 0.9103, Training time: 3.6940 seconds
Epoch 4, Loss: 0.2629, Val Acc: 0.9135, Training time: 3.5930 seconds
Epoch 5, Loss: 0.2488, Val Acc: 0.9146, Training time: 3.6040 seconds
Epoch 

Epoch 0, Loss: 0.5798, Val Acc: 0.9034, Training time: 5.8220 seconds
Epoch 1, Loss: 0.2720, Val Acc: 0.9160, Training time: 5.5250 seconds
Epoch 2, Loss: 0.2321, Val Acc: 0.9232, Training time: 5.4240 seconds
Epoch 3, Loss: 0.2143, Val Acc: 0.9259, Training time: 5.3220 seconds
Epoch 4, Loss: 0.2039, Val Acc: 0.9257, Training time: 5.4650 seconds
Epoch 5, Loss: 0.1964, Val Acc: 0.9279, Training time: 5.5080 seconds
Epoch 6, Loss: 0.1892, Val Acc: 0.9298, Training time: 5.3540 seconds
Epoch 7, Loss: 0.1845, Val Acc: 0.9292, Training time: 5.3170 seconds
Epoch 8, Loss: 0.1884, Val Acc: 0.9303, Training time: 5.2880 seconds
Epoch 9, Loss: 0.1729, Val Acc: 0.9282, Training time: 5.2420 seconds
Epoch 10, Loss: 0.1700, Val Acc: 0.9322, Training time: 5.3370 seconds
Epoch 11, Loss: 0.1662, Val Acc: 0.9321, Training time: 5.3220 seconds
Epoch 12, Loss: 0.1640, Val Acc: 0.9323, Training time: 5.3160 seconds
Epoch 13, Loss: 0.1628, Val Acc: 0.9333, Training time: 5.3290 seconds
Epoch 14, Loss: 

Epoch 6, Loss: 0.1907, Val Acc: 0.9285, Training time: 6.1080 seconds
Epoch 7, Loss: 0.1849, Val Acc: 0.9284, Training time: 6.0670 seconds
Epoch 8, Loss: 0.1805, Val Acc: 0.9295, Training time: 6.0500 seconds
Epoch 9, Loss: 0.1728, Val Acc: 0.9311, Training time: 6.4030 seconds
Epoch 10, Loss: 0.1688, Val Acc: 0.9316, Training time: 6.1110 seconds
Epoch 11, Loss: 0.1632, Val Acc: 0.9328, Training time: 6.1370 seconds
Epoch 12, Loss: 0.1630, Val Acc: 0.9322, Training time: 6.0680 seconds
Epoch 13, Loss: 0.1603, Val Acc: 0.9327, Training time: 6.2010 seconds
Epoch 14, Loss: 0.1552, Val Acc: 0.9332, Training time: 6.0690 seconds
Epoch 15, Loss: 0.1486, Val Acc: 0.9345, Training time: 6.1880 seconds
Epoch 16, Loss: 0.1467, Val Acc: 0.9349, Training time: 6.2930 seconds
Epoch 17, Loss: 0.1422, Val Acc: 0.9346, Training time: 6.1030 seconds
Epoch 18, Loss: 0.1418, Val Acc: 0.9346, Training time: 6.0600 seconds
Epoch 19, Loss: 0.1390, Val Acc: 0.9350, Training time: 6.1280 seconds
Total Time

In [35]:
import torch
import numpy as np
from dgl.dataloading import GraphDataLoader
from sklearn.metrics import precision_recall_fscore_support


def evaluate_model(model, dgl_graphs):
    """
    Evaluate the provided model on the given graphs.
    
    Parameters:
    model (nn.Module): The model to be evaluated.
    dgl_graphs (list): A list of DGL Graph objects to be used for evaluation.
    
    Returns:
    dict: A dictionary containing the Accuracy, Precision, Recall, and F1 Score of the model on the test data.
    """
    # Initialization of lists to store the true labels and the predicted labels
    all_labels = []
    all_preds = []
    total_correct = 0
    total_samples = 0
    
    # Set the model to evaluation mode
    model.eval()
    
    # Define the test data loader
    test_dataloader = GraphDataLoader(dgl_graphs, batch_size=32, shuffle=False)
    
    # Iterate over the test data
    for batched_graph in test_dataloader:
        # Extract features, labels, and the test mask from the batched graph
        features = batched_graph.ndata['feat']
        labels = batched_graph.ndata['label']
        test_mask = batched_graph.ndata['test_mask']
        
        # Perform forward pass through the model to get the logits
        logits = model(batched_graph, features)
        
        # Get predictions
        _, predicted = torch.max(logits, 1)
        
        # Update the correct predictions count and total samples count
        total_correct += (predicted == labels).float().sum().item()
        total_samples += len(labels)
        
        # Store the true and predicted labels
        all_labels.append(labels[test_mask].cpu().numpy())
        all_preds.append(predicted[test_mask].cpu().numpy())
    
    # Convert the list of true and predicted labels to NumPy arrays
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    
    # Compute Precision, Recall, and F1 Score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    test_acc = total_correct / total_samples
    
    # Return the computed metrics as a dictionary
    return {
        'Test Accuracy': test_acc,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }


In [37]:
# Create a model instance with the same structure as the saved model
model = SAGEModel(in_feats=7, num_classes=9, params=best_params)  

# Load the model weights
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model and print the results
results = evaluate_model(model, dgl_graphs)
print(f"Test Accuracy: {results['Test Accuracy']:.4f}")
print(f"Precision: {results['Precision']:.4f}")
print(f"Recall: {results['Recall']:.4f}")
print(f"F1 Score: {results['F1 Score']:.4f}")

Test Accuracy: 0.9474
Precision: 0.8655
Recall: 0.7990
F1 Score: 0.8148
